In [1]:
import random
import pandas as pd
from time import sleep
from threading import Thread
from lxml import etree
import requests
from snownlp import SnowNLP
import time
import numpy as np

In [2]:
boxings = pd.read_table('boxings.txt', header=None)

/home/eloise/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [3]:
boxings = boxings.dropna(how='any')

In [4]:
input_file = 'ratings.txt'
input_fobj = open(input_file, 'r')
content = input_fobj.readlines()
input_fobj.close()

In [5]:
name = [] 
rating = [] 
id_number = []

for i in range(len(content)):
    data = content[i].strip().split('\t')

    data[1] = float(data[1])
    if data[1] == 0:
        continue

    name.append(data[0])
    rating.append(data[1])
    id_number.append(data[2])

In [6]:
names = pd.Series(name)
ratings = pd.Series(rating)
ids = pd.Series(id_number)
df = pd.concat([names, ratings, ids], axis=1)

In [7]:
boxings.columns = ['name', 'boxing']
df.columns = ['name', 'rating', 'id']

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1931 entries, 0 to 1930
Data columns (total 3 columns):
name      1931 non-null object
rating    1931 non-null float64
id        1931 non-null object
dtypes: float64(1), object(2)
memory usage: 45.3+ KB


In [9]:
boxings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1411 entries, 6 to 1928
Data columns (total 2 columns):
name      1411 non-null object
boxing    1411 non-null object
dtypes: object(2)
memory usage: 33.1+ KB


In [10]:
boxings = boxings.set_index('name')
df = df.set_index('name')

In [11]:
df2 = pd.merge(boxings, df, how='inner', left_index=True, right_index=True, sort=True)
df2.sample(10)

,boxing,rating,id
name,,,
抢劫坚果店,3637.8万,6.5,25817617
狙击时刻,168.4万,4.2,25962762
猪太狼的夏天,28.2万,6.7,26877504
黄石的孩子,1233.2万,6.7,1923474
蝴蝶飞,702.0万,5.5,2359442
左右,142.4万,6.5,2059225
007：大破量子危机,12075.1万,6.4,1946882
A面B面,401.8万,5.7,3426848
冰雪大作战,4319.7万,6.1,26681656


In [12]:
headers = { "Accept":"text/html,application/xhtml+xml,application/xml;",
            "Accept-Encoding":"gzip",
            "Referer":"http://www.example.com/" }

user_agent_list = [
 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) '
  'Chrome/45.0.2454.85 Safari/537.36 115Browser/6.0.3',
 'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50',
 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50',
 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)',
 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)',
 'Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1',
 'Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11',
 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SE 2.X MetaSr 1.0; SE 2.X MetaSr 1.0; .NET CLR 2.0.50727; SE 2.X MetaSr 1.0)',
 'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0',
 'Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1',
]

def get_req(search_url):
    user_agent = random.choice(user_agent_list)
    headers['User-Agent'] = user_agent
    
    wait_sec = random.random()*2
    time.sleep(wait_sec)
    searchHtml = requests.get(search_url, headers = headers)
    
    return searchHtml

In [13]:
def fetch_weighted_rating(id_number):
    url = 'https://movie.douban.com/subject/%s/?from=showing' % id_number

    req = get_req(url)
    html = req.text
    tree = etree.HTML(html)

    try:
        director.append(tree.xpath('//*[@id="info"]/span[1]/span[2]/a/text()'))
    except:
        director.append(np.nan)
    
    try:
        scriptwriter.append(tree.xpath("///div[@id='info']/span[2]/span[2]/a/text()"))
    except:
        scriptwriter.append(np.nan)
    
    try:
        starring.append(tree.xpath("//div[@id='info']/span[3]/span[2]/a[position()<6]/text()"))
    except:
        starring.append(np.nan)
    
    try:
        genre.append(tree.xpath("//div[@id='info']/span[@property='v:genre']/text()")[0])
    except:
        genre.append(np.nan)
    
    try:
        release_date.append(tree.xpath("//div[@id='info']/span[@property='v:initialReleaseDate']/text()")[0])
    except:
        release_date.append(np.nan)
        
    try:
        total_people.append(tree.xpath('//span[@property="v:votes"]/text()')[0])
    except:
        total_people.append(np.nan)
        
    try:
        ratings_weight.append(tree.xpath('//span[@class="rating_per"]/text()'))
    except:
        ratings_weight.append(np.nan)
        
    try:
        running_time.append(tree.xpath("//div[@id='info']/span[@property='v:runtime']/@content")[0])
    except:
        running_time.append(np.nan)
        
    try:
        total_people.append(tree.xpath('//span[@property="v:votes"]/text()')[0])
    except:
        total_people.append(np.nan)
        
    try:
        comment = tree.xpath( "//div[@class ='comment']/p/span/text()")
        s = SnowNLP('。'.join(comment))
        sentimentslist.append(s.sentiments)
    except:
        sentimentslist.append(np.nan)


In [14]:
sentimentslist = []
director = []
scriptwriter = []
starring = []
genre = []
release_date = []
total_people = []
ratings_weight = []
running_time = []

for i in id_number:
    fetch_weighted_rating(i)

In [20]:
senti = pd.Series(sentimentslist)
dirct = pd.Series(director)
script = pd.Series(scriptwriter)
star = pd.Series(starring)
genr = pd.Series(genre)
date = pd.Series(release_date)
people = pd.Series(total_people)
running = pd.Series(running_time)


In [21]:
df3 = pd.concat([names, ratings, ids, senti, dirct, script, star, genr, date, people, running], axis=1)

In [23]:
df3.columns = ['name', 'rating', 'id', 'sentiment', 'director', 'scriptwriter', 'starring', 'genre', 'date', 'total_people', 'running_time']
df3.sample(10)

,name,rating,id,sentiment,director,scriptwriter,starring,genre,date,total_people,running_time
198,伤城,6.8,1867408,0.996946,"[刘伟强, 麦兆辉]","[庄文强, 麦兆辉]","[梁朝伟, 金城武, 徐静蕾, 舒淇, 杜汶泽]",剧情,2006-12-21(香港),101,110
3078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1268,NaN
134,鲨鱼故事,7.1,1308757,1.000000,"[罗伯·莱特曼, 维基·詹森, Bibo Bergeron]","[迈克尔·J·威尔森, 罗伯·莱特曼]","[威尔·史密斯, 罗伯特·德尼罗, 蕾妮·齐薇格, 安吉丽娜·朱莉, 杰克·布莱克]",喜剧,2005-04-29(中国大陆),1846,90
78,三十八度,6.2,1476912,1.000000,"[刘新, 艾颖]","[朱苹, 刘新]","[黄磊, 陶红, 谢雨欣, 刘龙, 高远]",喜剧,2003-09-04(中国大陆),86,100
1385,搏击迷城,3.9,25896040,0.999997,[陈丽英],"[陈丽英, 林纪陶]","[陈伟霆, 赵柯, 李菲儿, 渡边奈绪子, 斯琴高娃]",动作,2015-06-05(中国大陆),46318,98
3747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,793,NaN
3128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4341,NaN
1800,一念无明,7.9,26704590,0.951631,[黄进],"[黄进, 陈楚珩]","[曾志伟, 余文乐, 方皓玟, 金燕玲, 陈学文]",剧情,2017-04-07(中国大陆),1947,101
3708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,176025,NaN
1927,嘉年华,8.2,27019527,1.000000,[文晏],[文晏],"[文淇, 周美君, 史可, 耿乐, 刘威葳]",剧情,2017-11-24(中国大陆),461939,107


In [30]:
df3 = df3.dropna(how='any')
df3.shape

(1722, 11)

In [32]:
df3.sample(10)

,name,rating,id,sentiment,director,scriptwriter,starring,genre,date,total_people,running_time
802,死亡迷局,3.6,5342081,0.802879,[鲁坚],"[王建武, 鲁坚]","[董娉, 傅冲, 阮惟新]",剧情,2012-03-23(中国大陆),578,92
1058,模特魅影,2.8,24381046,0.999696,[佟秀玄],[佟秀玄],"[倪慕斯, 杨骏, 唐宁, 李星海, 饶薇]",剧情,2013-07-26(中国大陆),75,108
277,远山,8.6,2079735,1.000000,[胡杰],[],[],纪录片,1995,651,40
1179,巴啦啦小魔仙之魔法的考验,3.3,25791073,0.957587,"[于人, 刘浩]","[齐圆, 李冠萱]","[陈雨薇, 赵今麦, 刘黛希, 王慧, 徐可]",儿童,2014-01-23(中国大陆),4456,90
185,图雅的婚事,8.0,1966460,1.000000,[王全安],"[芦苇, 王全安]","[余男, 巴特尔, 森格]",剧情,2006-09-21(中国大陆),107,95
1322,王牌,4.6,25743807,1.000000,[范建浍],"[范建浍, 陈建忠, 吴放]","[林志玲, 梁家辉, 钟欣潼, 蒙亭宜, 邬君梅]",剧情,2014-12-31(中国大陆),379874,90
554,叶问前传,5.7,4134631,0.999989,[邱礼涛],[李敏],"[杜宇航, 樊少皇, 元彪, 陈嘉桓, 黄奕]",动作,2010-06-24(香港),972,100
1809,蓝精灵：寻找神秘村,6.3,19899718,1.000000,[凯利·阿斯博瑞],"[帕米拉·里本, 斯泰西·哈曼, 佩约]","[黛米·洛瓦托, 雷恩·威尔森, 乔·曼根尼罗, 杰克·麦克布瑞尔, 丹尼·朴迪]",喜剧,2017-04-21(中国大陆),1087,90
866,边境风云,7.7,11501396,1.000000,[程耳],[程耳],"[孙红雷, 王珞丹, 张默, 杨坤, 倪大红]",剧情,2012-08-17(中国大陆),4553,100
1390,我的男友和狗,5.7,24851526,1.000000,[陈慧翎],"[叶​乃​菁​, 原田舞叶]","[张钧甯, 何润东, 赖佩霞, 丁春诚, 薛仕凌]",爱情,2015-06-19(中国大陆),8641,94


In [33]:
df3.to_csv('douban.csv')